the idea is to use the "istypeof" info. specifically, adding relations by claiming all subtypes should have the same feature as supconcept. equivalent to making deduction (i.e. a demonstration of why logic is useful).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # delete later
import random
import math
from IPython.core.debugger import set_trace
import pickle

In [2]:
#[Data_onehot,Data,IndexDict] = pickle.load(open('tableB1-entities','rb'))
[Data,IndexDict,idsubdict]=pickle.load(open('../data/mammal_data.p','rb'))

In [16]:
pddict={}
for key in IndexDict.keys():
    pddict[key] = pd.Series(IndexDict[key]) 

In [17]:
IndexDict = pddict

In [47]:
def idx2name(triplets):
    # input: dictionary with h,r,t,y each is a list.
    if isinstance(triplets, pd.DataFrame):
        triplets=triplets.to_dict('list')
    
    # output: string of each triplet in natural language
    ntrip=len(triplets['y'])
    ngoodtrip=sum(triplets['y']) 
    
    assert ngoodtrip<=ntrip
        
    alltrips = ['' for k in range(ntrip)]
    goodtrips = []
    for ii in range(ntrip):
        alltrips[ii]+=IndexDict['entity'][triplets['h'][ii]] + '-'
        alltrips[ii]+=IndexDict['relation'][triplets['r'][ii]] + '-'
        alltrips[ii]+=IndexDict['entity'][triplets['t'][ii]]
        
        if triplets['y'][ii]:
            goodtrips.append(alltrips[ii])
        alltrips[ii]+= ': '+str(bool(triplets['y'][ii]>0))
    return alltrips,goodtrips  

In [53]:
# identifying sup / sub concepts
# simple version: 1 layer of deduction // TODO: 2 layered propagation

# 1. find all correct triplets with "iskindof" relation

rid=np.where(IndexDict['relation']=='is a kind of')[0][0]

corrD = Data[Data['y']==1]
goodD=corrD[corrD['r']==rid] 
subcon = {}
for kt in set(goodD['t']):
    subcon[kt]=list(goodD[goodD['t']==kt]['h'])
    
# 2. add all triplets with head of the sup concept
newdlist=[]
for supc in subcon.keys():
    tripid = list(Data['h'][Data['h']==supc].index)
    for sub in subcon[supc]:        
        for ii in tripid:
            newdlist.append([sub,Data['r'][ii],Data['t'][ii]])
newdata = pd.DataFrame(data=newdlist,columns=['h','r','t'])
newdata['y']=np.ones(len(newdlist),dtype=int)

In [55]:
print(idx2name(newdata[112:122])[1]) #sanity check to see if new 

['adult raccoon-is eaten by-bobcat', 'adult raccoon-is eaten by-coyote', 'adult raccoon-is eaten by-great horned owl', 'adult raccoon-is eaten by-red fox', 'adult raccoon-lives in-barrel', 'adult raccoon-lives in-burrow', 'adult raccoon-lives in-cave', 'adult raccoon-lives in-culvert', 'adult raccoon-lives in-den', 'adult raccoon-lives in-drain pipe']


In [63]:
Data=Data.append(newdata)

In [64]:
len(Data)

5035

In [67]:
h = Data['h'].iloc[ktrip]

In [70]:
# generate negative samples
nbadmax=2
new={}
new['r'] = []
new['h'] =[]
new['t'] = []

for ktrip in range(1,len(Data)):
    h = Data['h'].iloc[ktrip]
    r = Data['r'].iloc[ktrip]
    if (r==Data['r'].iloc[ktrip-1]) & (h==Data['h'].iloc[ktrip-1]):
        continue # repetitve head-relation
    else:
        tdict=Data['t']
        goodts = tdict[(Data['h']==h) & (Data['r']==r) & (Data['y']==1)]
        if len(goodts)==0:
            continue
        otherts = tdict[(Data['h']!=h) & (Data['r']==r)& (Data['y']==1)]
        badts=list(set(otherts)-set(goodts))
        if len(badts)!=0:
            nbad=min(nbadmax,len(badts))
            new['h'] = new['h']+[h for k in range(nbad)]
            new['r'] = new['r']+[r for k in range(nbad)]
            [new['t'].append(k) for k in np.random.choice(badts,size=nbad,replace=False)]
            
assert len(new['h'])==len(new['t'])

new['y'] = [-1 for k in range(len(new['t']))]

In [72]:
new=pd.DataFrame(new)

In [73]:
new

,h,r,t,y
0,7,2,10,-1
1,9,2,8,-1
2,11,3,30,-1
3,11,3,42,-1
4,13,3,12,-1
5,13,3,17,-1
6,15,3,42,-1
7,15,3,12,-1
8,16,3,12,-1
9,16,3,34,-1


In [74]:
MoreD=Data.append(new)

In [75]:
pickle.dump([MoreD,pddict,idsubdict], open('../data/mammal_data_addtype.p','wb'))